In [ ]:
import metric
import matplotlib.pyplot as plt
from scipy.optimize import newton
from diff_eqs_coeffs import *
from scipy.interpolate import interp1d
# from scipy.interpolate import bisplrep
# from scipy.interpolate import bisplev
# from scipy.interpolate import RBFInterpolator
# from scipy.interpolate import SmoothBivariateSpline
from scipy.interpolate import RectBivariateSpline
# from min_theta_calc import theta_min_calculator
from scipy.interpolate import griddata
# from scipy.spatial import KDTree
import json
from epsilon_computer import *



In [ ]:
def to_obtain_cusp(r, M, a, l, s0, gamma, kappa):
	theta = np.pi/2
	return metric.F0_0(kappa, gamma)*drlogut.drlogut_f(M, a, r, theta, l)-F1ast.f1ast_f(M, a, r, theta, l, s0, kappa, gamma)-Gast.gast_f(M, a, r, theta, l, s0, kappa, gamma)


def r_mb(M, a):
	return 2*M*(1-a/2+np.sqrt(1-a))

def radial_epsilon(r, epsilon, M, a, l, s0, kappa, gamma):
	return F0(epsilon, kappa, gamma)*drlogut.drlogut_f(M, a, r, np.pi/2, l) - F1ast.f1ast_f(M, a, r, np.pi/2, l, s0, kappa, gamma)-Gast.gast_f(M, a, r, np.pi/2, l, s0, kappa, gamma)

with open("parameters.json") as param_file:
    parameters = json.load(param_file)

M = float(parameters["M"])
a = float(parameters["a"])
l = float(parameters["l"])
s0 = float(parameters["s0"])
kappa = float(parameters["kappa"])
gamma = float(parameters["gamma"])
r_cusp_0 = r_mb(M, a)

r_cusp = newton(to_obtain_cusp, r_cusp_0, args=(M, a, l, s0, gamma, kappa), maxiter=1000)
print(r_cusp)

h = 0.005
imax = 30/h
current_epsilon = 1e-20
current_r = r_cusp
r_array = []
epsilon_array = []
i = 0


while i<imax:
	r_array.append(current_r)
	epsilon_array.append(current_epsilon)
	pre_r = current_r
	pre_epsilon = current_epsilon
	k1 = h*radial_epsilon(pre_r, pre_epsilon, M, a, l, s0, kappa, gamma)
	k2 = h*radial_epsilon(pre_r + h/2., pre_epsilon+k1/2., M, a, l, s0, kappa, gamma)
	k3 = h*radial_epsilon(pre_r + h/2., pre_epsilon+k2/2., M, a, l, s0, kappa, gamma)
	k4 = h*radial_epsilon(pre_r + h, pre_epsilon+k3, M, a, l, s0, kappa, gamma)
	current_epsilon = pre_epsilon + (1./6.)*(k1+2*k2+2*k3+k4)
	current_r = pre_r + h
	i = i + 1
	if (current_epsilon<1e-20):
		break

epsilon_array = np.asarray(epsilon_array)
r_array = np.asarray(r_array)

epsilon_max = epsilon_array.max()
r_max = r_array[np.where(epsilon_array == epsilon_max)]

r_out = r_array[-1]
print(r_out)
print(r_max, epsilon_max)

plt.plot(r_array, epsilon_array, c = 'black', linewidth = 3)
# plt.show()
plt.close()

boundary_array_k1 = epsilon_computer_f(M, a, l, s0, kappa, gamma, True, 0, 0)
boundary_y_array_k1 = boundary_array_k1[:,1]
boundary_x_array_k1 = boundary_array_k1[:,0]
y_max = 1.10*np.max(boundary_y_array_k1)
# y_max = 1.1*max_locations[1]
# theta_min = np.arctan2(r_max, y_max)

# plt.plot(boundary_x_array_k1, boundary_y_array_k1, c = 'black', linewidth = 3)
# plt.show()
# plt.close()

######## with a correction factor for y of 1.10
##This value could affect an analysis based only on the shape of the characteristics of k, be careful!!!!
boundary_func = interp1d(boundary_x_array_k1, 1.10*boundary_y_array_k1, kind='cubic')
########


x_precision = 0.025
y_precision = 0.005

In [ ]:
k0 = 1.

N_x = int((r_out - r_cusp)/x_precision)
N_y = int((y_max)/y_precision)

x_eq_array = np.linspace(r_cusp + x_precision, r_out, num=N_x, endpoint=False)
print(x_eq_array)
print(len(x_eq_array))
y_search_array = np.linspace(0, y_max, num=30*N_y)

initial_data_list = []

i = 0
i_max = len(x_eq_array)

while i < i_max:
	initial_data_list.append([x_eq_array[i],1e-10])
	i = i + 1


i = 0
h = y_precision

curves_grid = []

print("computing characteristics")


for initial_point in initial_data_list:
	current_x = initial_point[0]
	current_y = initial_point[1]
	current_curve_array = []
	current_curve_array.append([current_x, current_y])
	current_curve_x = []
	current_curve_y = []
	current_curve_x.append(current_x)
	current_curve_y.append(current_y)
	current_theta = np.pi/2

	while (current_y < boundary_func(current_x)):
		try:
			k1 = Ak(M, a, current_x, current_y, l, s0, kappa, gamma)
		except ValueError:
			current_curve_array = np.asarray(current_curve_array)
			curves_grid.append(current_curve_array)
			plt.plot(current_curve_x, current_curve_y, color=[np.random.random_sample(),np.random.random_sample(),np.random.random_sample()])
			break
		try:
			k2 = Ak(M, a, current_x + (1./2.)*(k1*h), current_y + (1./2.)*h, l, s0, kappa, gamma)
		except ValueError:
			current_curve_array = np.asarray(current_curve_array)
			curves_grid.append(current_curve_array)
			plt.plot(current_curve_x, current_curve_y, color=[np.random.random_sample(),np.random.random_sample(),np.random.random_sample()])
			break
		try:
			k3 = Ak(M, a, current_x + (1./2.)*(k2*h), current_y + (1./2.)*h, l, s0, kappa, gamma)
		except ValueError:
			current_curve_array = np.asarray(current_curve_array)
			curves_grid.append(current_curve_array)
			plt.plot(current_curve_x, current_curve_y, color=[np.random.random_sample(),np.random.random_sample(),np.random.random_sample()])
			break
		try:
			k4 = Ak(M, a, current_x + (k3*h), current_y + h, l, s0, kappa, gamma)
		except ValueError:
			current_curve_array = np.asarray(current_curve_array)
			curves_grid.append(current_curve_array)
			plt.plot(current_curve_x, current_curve_y, color=[np.random.random_sample(),np.random.random_sample(),np.random.random_sample()])
			break
		if (k1 != k1 or k2 != k2 or k3 != k3 or k4 != k4):
				current_curve_array = np.asarray(current_curve_array)
				curves_grid.append(current_curve_array)
				plt.plot(current_curve_x, current_curve_y, color=[np.random.random_sample(),np.random.random_sample(),np.random.random_sample()])
				break

		current_x = current_x + (h/6.)*(k1 + 2*k2 + 2*k3 + k4)
		current_y = current_y + h
		current_theta = np.arctan2(current_x, current_y)
		current_curve_array.append([current_x, current_y])
		current_curve_x.append(current_x)
		current_curve_y.append(current_y)

	else:
		current_curve_array = np.asarray(current_curve_array)
		curves_grid.append(current_curve_array)
		plt.plot(current_curve_x, current_curve_y, color=[np.random.random_sample(),np.random.random_sample(),np.random.random_sample()])

	print(i)
	i = i + 1

curves_grid = np.asarray(curves_grid)

# np.savetxt("curves_grid_0.dat", curves_grid[0])
# np.savetxt("curves_grid_1.dat", curves_grid[1])
# np.savetxt("curves_grid_2.dat", curves_grid[2])


plt.show()
plt.close()
